<h1 align='center'> <b>Data Analysis & Stats</b> </h1>

## ***Libreries***

- <a href="https://pandas.pydata.org/"><img src="https://pandas.pydata.org/docs/_static/pandas.svg" style='height:70px'/></a>
- <a href="https://matplotlib.org/"><img src="https://matplotlib.org/stable/_static/logo2.svg" style='height:50px'/></a>


## ***Language***


- <a href="https://www.python.org/"><img src="https://www.python.org/static/img/python-logo.png" style='height:50px'>

In [15]:
import pandas as pd     
import os 

## Obtengo la ruta donde se almacena el archivo CSV

In [16]:
EVENTS_PATH =  os.path.join(os.path.abspath('..'),'src','database','events','events.csv') 
EVENTS_PATH

'c:\\Users\\galla\\Documents\\Facultad\\Integrador\\grupo27\\src\\database\\events\\events.csv'

## Leo el archivo y lo convierto en Data Frame 

In [17]:
data_frame = pd.read_csv(EVENTS_PATH,encoding='utf-8',index_col='timestamp')
data_frame.head() 

,id,evento,personas a adivinar,usuarie,estado,texto ingresado,respuesta,nivel
timestamp,,,,,,,,
1655329225,cdc08059179d455881d0d78846f03ad2,inicio_partida,10,Josu,-,-,-,Intermedio
1655329226,f72b677ca19144bc837bc11e32cfb000,intento,10,Josu,ok,César Huanca,Aron Dønnum,Intermedio
1655329227,aab6718268dd466fbf7f97d79abdee24,intento,10,Josu,error,Timothée Dieng,Aron Dønnum,Intermedio
1655329228,a3d232a6efb64a5787e24c0cd8277a20,intento,10,Josu,error,Duško Tošic,Loïck Landre,Intermedio
1655329229,3f29d49c83d9446780fb9fefb5d1785c,intento,10,Josu,ok,Loïck Landre,Cristian González,Intermedio
